In [2]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('../../../../last4years.csv')[['speakers', 'speeches', 'speech_id', 'person_id', 'time_of_speech', 'debate_id']]

In [258]:
headings = pd.read_csv('all_headings.csv')

This is where I used the headings as bins, created unique bin edges and then assigned each speech the correct topic. 

In [43]:
for i in range(len(headings.id)):
    headings['id'][i] += i/100000

/Users/alexdcruz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [133]:
def slicer(df):
    bins = headings[headings.debate_date.isin(df.debate_id)].id.values
    labels = headings[headings.debate_date.isin(df.debate_id)]['Unnamed: 0'].values
    bins = bins.tolist() + [1000000]
    df['topic'] = pd.cut(df.speech_id, bins = bins, labels = labels)
    return df

In [135]:
new_df = df.groupby('debate_id').apply(slicer)

In [139]:
headings.drop(columns = 'Unnamed: 0.1', inplace = True)

In [142]:
headings.columns = ['topic', 'headings', 'id', 'debate_date']

In [148]:
new_df_2 = new_df.merge(headings, how = 'left', on = 'topic')

In [152]:
new_df_2.drop(columns = ['debate_date', 'id', 'time_of_speech', 'topic', 'speech_id'], inplace = True)

In [153]:
new_df_2

,speakers,speeches,person_id,debate_id,headings
0,Brandon Lewis,Let me finish the point. It takes five to 10 m...,uk.org.publicwhip/person/24879,2020-01-07a,NaN
1,Brandon Lewis,I will be brief; I just want to respond to a c...,uk.org.publicwhip/person/24879,2020-01-07a,NaN
2,Lindsay Hoyle,"On behalf of the whole House, I wish to expres...",uk.org.publicwhip/person/10295,2020-01-07b,Speaker’s Statement
3,NaN,The Chancellor of the Exchequer was asked—,NaN,2020-01-07b,Treasury
4,Mark Logan,Whether he plans to increase the level of per ...,uk.org.publicwhip/person/25886,2020-01-07b,Per Pupil Funding
...,...,...,...,...,...
150707,Bob Stewart,The one thing that the hon. Gentleman has not ...,uk.org.publicwhip/person/24907,2016-12-20b,WOOLWICH BARRACKS
150708,Matthew Pennycook,Absolutely. The hon. Gentleman may know that t...,uk.org.publicwhip/person/25379,2016-12-20b,WOOLWICH BARRACKS
150709,Mark Lancaster,I congratulate the hon. Member for Greenwich a...,uk.org.publicwhip/person/11715,2016-12-20b,WOOLWICH BARRACKS
150710,Bob Stewart,I accept everything that the Minister has said...,uk.org.publicwhip/person/24907,2016-12-20b,WOOLWICH BARRACKS


In [157]:
new_df_2.person_id = new_df_2.person_id.str.replace('uk.org.publicwhip/person/', '')

In [213]:
new_df_2.person_id = pd.to_numeric(new_df_2.person_id, errors = 'coerce')

In [225]:
#new_df_2.dropna(subset = ['person_id'], inplace = True)

In [230]:
#new_df_2 = new_df_2.astype({'person_id': 'int32'})

In [266]:
mps = pd.read_csv('mps.csv')
mps_2017 = pd.read_csv('mps_2017.csv')
mps_2015 = pd.read_csv('mps_2015.csv')
mps = mps.append(mps_2017, ignore_index = True)
mps = mps.append(mps_2015, ignore_index = True)
mps.drop_duplicates(inplace = True)

In [268]:
mps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915 entries, 0 to 1946
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Person ID     915 non-null    int64 
 1   First name    915 non-null    object
 2   Last name     915 non-null    object
 3   Party         915 non-null    object
 4   Constituency  915 non-null    object
 5   URI           915 non-null    object
dtypes: int64(1), object(5)
memory usage: 50.0+ KB


In [269]:
parties = mps[['Person ID', 'Party', 'Constituency']]
parties.columns = ['person_id', 'party', 'constituency']

In [271]:
final_df = new_df_2.merge(parties, how = 'left', on = 'person_id')

In [272]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147459 entries, 0 to 147458
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speakers      147459 non-null  object
 1   speeches      147438 non-null  object
 2   person_id     147459 non-null  int32 
 3   debate_id     147459 non-null  object
 4   headings      145430 non-null  object
 5   party         147414 non-null  object
 6   constituency  147414 non-null  object
dtypes: int32(1), object(6)
memory usage: 8.4+ MB


In [273]:
final_df[final_df.party.isnull()].speakers.unique()

array(['Tan Dhesi', 'Robert Alexander Courts', 'Jack Edgar Brereton',
       'Robert Courts', 'Jane Dodds', 'Lisa Forbes', 'Ross Thomson',
       'Emma Little-Pengelly'], dtype=object)

In [286]:
final_df.headings.fillna('None', inplace = True)

In [293]:
final_df[final_df.party.isnull()]

,speakers,speeches,person_id,debate_id,headings,party,constituency
959,Tan Dhesi,"For some weird, inexplicable reason, the Gover...",25857,2020-01-14b,Victims of Domestic Abuse,NaN,NaN
1063,Robert Alexander Courts,Will the Foreign Secretary outline the steps t...,25840,2020-01-14b,Joint Comprehensive Plan Of Action,NaN,NaN
1198,Robert Alexander Courts,It is an honour to speak towards the end of th...,25840,2020-01-14b,Education and Local Government,NaN,NaN
1264,Tan Dhesi,What steps he is taking to increase (a) recrui...,25857,2020-01-20e,Childminders: Recruitment and Retention,NaN,NaN
1266,Tan Dhesi,The Early Years Alliance has stated that the G...,25857,2020-01-20e,Childminders: Recruitment and Retention,NaN,NaN
1578,Jack Edgar Brereton,As office costs are substantially cheaper in S...,25899,2020-01-22b,None,NaN,NaN
1595,Robert Courts,I welcome the manifesto commitment to offer gu...,25840,2020-01-22d,Civil Service Reform,NaN,NaN
1984,Jack Edgar Brereton,I join my hon. Friend the Member for Newcastle...,25899,2020-01-23a,Most Favoured Nation Tariffs,NaN,NaN
2317,Tan Dhesi,Does my right hon. Friend agree that violent c...,25857,2020-01-29b,None,NaN,NaN
2511,Robert Courts,We are dealing with one of the world’s longest...,25840,2020-01-30b,Middle East Peace Plan,NaN,NaN


In [338]:
final_df.party.unique()

array(['Conservative', 'Speaker', 'Labour', 'Liberal Democrat',
       'Scottish National Party', 'Labour/Co-operative', 'DUP',
       'Independent', 'Green', 'Plaid Cymru', 'Alliance',
       'Social Democratic and Labour Party', 'UUP', 'UKIP'], dtype=object)

In [331]:
def fill_parties(row):
    if row.speakers == 'Tan Dhesi' or row.speakers == 'Lisa Forbes':
        return  'Labour'
    if (row.speakers == 'Robert Alexander Courts' or row.speakers == 'Robert Courts' 
        or row.speakers == 'Jack Edgar Brereton' or row.speakers == 'Ross Thomson'):
        return 'Conservative'
    if row.speakers == 'Jane Dodds':
        return 'Liberal Democrat'
    if row.speakers == 'Emma Little-Pengelly':
        return 'DUP'
    else:
        return row.party

In [332]:
final_df.party = final_df.apply(fill_parties, axis = 1)

In [333]:
def fill_constituency(row):
    if row.speakers == 'Tan Dhesi':
        return 'Slough'
    if row.speakers == 'Lisa Forbes':
        return  'Peterborough'
    if row.speakers == 'Robert Alexander Courts' or row.speakers == 'Robert Courts':
        return 'Whitney'
    if row.speakers == 'Jack Edgar Brereton':
        return 'Stoke-on-Trent South'
    if row.speakers == 'Ross Thomson':
        return 'Aberdeen South'
    if row.speakers == 'Jane Dodds':
        return 'Brecon and Radnorshire'
    if row.speakers == 'Emma Little-Pengelly':
        return 'Belfast South'
    else:
        return row.constituency

In [334]:
final_df.constituency = final_df.apply(fill_constituency, axis = 1)

In [335]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147459 entries, 0 to 147458
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speakers      147459 non-null  object
 1   speeches      147438 non-null  object
 2   person_id     147459 non-null  int32 
 3   debate_id     147459 non-null  object
 4   headings      147459 non-null  object
 5   party         147459 non-null  object
 6   constituency  147459 non-null  object
dtypes: int32(1), object(6)
memory usage: 8.4+ MB


In [340]:
final_df.dropna(inplace = True)

In [345]:
from datetime import datetime

In [348]:
final_df.debate_id[:-1]

0         2020-01-07a
1         2020-01-07a
2         2020-01-07b
3         2020-01-07b
4         2020-01-07b
             ...     
147453    2016-12-20b
147454    2016-12-20b
147455    2016-12-20b
147456    2016-12-20b
147457    2016-12-20b
Name: debate_id, Length: 147437, dtype: object

In [359]:
dates = []
for cell in final_df['debate_id']:
    dates.append(cell[:-1])
final_df['dates'] = dates

In [361]:
final_df.dates = pd.to_datetime(final_df.dates)

In [363]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147438 entries, 0 to 147458
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   speakers      147438 non-null  object        
 1   speeches      147438 non-null  object        
 2   person_id     147438 non-null  int32         
 3   debate_id     147438 non-null  object        
 4   headings      147438 non-null  object        
 5   party         147438 non-null  object        
 6   constituency  147438 non-null  object        
 7   dates         147438 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(6)
memory usage: 14.6+ MB


In [365]:
final_df.party.value_counts()

Conservative                          85955
Labour                                32290
Scottish National Party               10171
Speaker                                6299
Labour/Co-operative                    5579
Liberal Democrat                       2722
DUP                                    2186
Independent                             800
Plaid Cymru                             719
Green                                   311
Social Democratic and Labour Party      231
UUP                                     129
Alliance                                 23
UKIP                                     23
Name: party, dtype: int64

In [366]:
final_df[final_df.party == 'Labour/Co-operative']

,speakers,speeches,person_id,debate_id,headings,party,constituency,dates
77,Anneliese Dodds,A very happy new year to you and everyone in t...,25618,2020-01-07b,Infrastructure Investment Distribution,Labour/Co-operative,Oxford East,2020-01-07
93,Barry Sheerman,Does the Minister agree that not only need mor...,10534,2020-01-07b,Street Crime,Labour/Co-operative,Huddersfield,2020-01-07
99,Meg Hillier,One of the major crises is the recruitment of ...,11605,2020-01-07b,Frontline Health Services,Labour/Co-operative,Hackney South and Shoreditch,2020-01-07
184,Stephen Doughty,The Secretary of State is right to call for de...,25166,2020-01-07b,Middle East: Security,Labour/Co-operative,Cardiff South and Penarth,2020-01-07
323,Lloyd Russell-Moyle,Does my hon. Friend agree that we only need to...,25667,2020-01-07b,Saving of ECA for implementation period,Labour/Co-operative,"Brighton, Kemptown",2020-01-07
...,...,...,...,...,...,...,...,...
147210,Meg Hillier,"As the Minister will know, the Public Accounts...",11605,2016-12-20b,NHS: Financial Recovery,Labour/Co-operative,Hackney South and Shoreditch,2016-12-20
147315,Christina Rees,"When the original bid was abandoned in 2011, t...",25332,2016-12-20b,SKY: 21ST CENTURY FOX TAKEOVER BID,Labour/Co-operative,Neath,2016-12-20
147328,Seema Malhotra,My constituents have also contacted me with th...,25150,2016-12-20b,SKY: 21ST CENTURY FOX TAKEOVER BID,Labour/Co-operative,Feltham and Heston,2016-12-20
147374,Gareth Thomas,I apologise to the House for being unable to b...,10594,2016-12-20b,LEASEHOLD AND COMMONHOLD REFORM,Labour/Co-operative,Harrow West,2016-12-20


In [368]:
final_df.to_csv('cleaned_speeches_and_party.csv')

In [371]:
tester = final_df[['speakers', 'party','constituency', 'dates', 'headings', 'speeches']]

In [375]:
speakers_group = tester.groupby(['speakers','party'])['headings'].count().reset_index()
speakers_group.columns=['speakers','party','speeches']
speakers_group.head()

,speakers,party,speeches
0,Aaron Bell,Conservative,32
1,Abena Oppong-Asare,Labour,18
2,Adam Afriyie,Conservative,36
3,Adam Holloway,Conservative,16
4,Adrian Bailey,Labour/Co-operative,83


In [380]:
import re

In [382]:
special_ident = re.compile(r"[^ -z]")
special_chars=[]
for i, row in final_df.iterrows():
    results = re.findall(special_ident,row['speeches'])
    if results: special_chars.append(results)
        

In [383]:
unique_chars = []
for chars in special_chars:
    for char in chars:
        if char not in unique_chars:
            unique_chars.append(char)
unique_chars

['—',
 '\n',
 '’',
 '£',
 '“',
 '”',
 'à',
 'ğ',
 'é',
 '‘',
 '\xa0',
 '…',
 'â',
 '€',
 '™',
 'ç',
 'á',
 'í',
 'ō',
 'ŵ',
 'Š',
 'ö',
 'ô',
 'š',
 'Ž',
 'ć',
 'ł',
 '½',
 '°',
 '–',
 'ü',
 'ŷ',
 'ó',
 'Ø',
 'ê',
 'ò',
 'ñ',
 '¸',
 'Â',
 'è',
 'č',
 'π',
 'º',
 'ï',
 'É',
 'æ',
 'ż',
 'å',
 '\u200b',
 'İ',
 'Ö',
 'Ş',
 'ş',
 'ä',
 'ú',
 'ë',
 'ń',
 'Ł',
 'ź',
 '|',
 'Ç',
 'Ó',
 '\u200f',
 'û',
 'Á',
 'ø',
 'ã',
 'ù',
 'ž',
 'Å',
 'µ',
 'ı',
 '\u2009',
 '\xad',
 '\u2003',
 '©',
 'î',
 'Æ',
 'Ð']

In [3]:
import pandas as pd
unordered = pd.read_csv('../part-02/cleaned_speeches_and_party.csv')

In [6]:
unordered[unordered.speakers == 'Nigel Huddleston']

,Unnamed: 0,speakers,speeches,person_id,debate_id,headings,party,constituency,dates
5001,5001,Nigel Huddleston,We want to get more women and girls embracing ...,25381,2020-03-11b,Women In Public Office,Conservative,Mid Worcestershire,2020-03-11
7791,7791,Nigel Huddleston,I thank the hon. Member for Swansea East (Caro...,25381,2020-03-19a,Gambling Advertising in Sport,Conservative,Mid Worcestershire,2020-03-19
7793,7793,Nigel Huddleston,I thank the hon. Member for his comments. He m...,25381,2020-03-19a,Gambling Advertising in Sport,Conservative,Mid Worcestershire,2020-03-19
7795,7795,Nigel Huddleston,"The hon. Lady is making a fair point and, as I...",25381,2020-03-19a,Gambling Advertising in Sport,Conservative,Mid Worcestershire,2020-03-19
7797,7797,Nigel Huddleston,The hon. Gentleman makes an important point. O...,25381,2020-03-19a,Gambling Advertising in Sport,Conservative,Mid Worcestershire,2020-03-19
...,...,...,...,...,...,...,...,...,...
143396,143414,Nigel Huddleston,The Secretary of State is right to point out t...,25381,2016-11-29b,CORPORATE GOVERNANCE,Conservative,Mid Worcestershire,2016-11-29
143621,143639,Nigel Huddleston,What recent discussions he has had with key st...,25381,2016-11-30b,THE SECRETARY OF STATE WAS ASKED—,Conservative,Mid Worcestershire,2016-11-30
143624,143642,Nigel Huddleston,Does the Secretary of State agree that we shou...,25381,2016-11-30b,THE SECRETARY OF STATE WAS ASKED—,Conservative,Mid Worcestershire,2016-11-30
145460,145478,Nigel Huddleston,What steps he is taking to encourage innovatio...,25381,2016-12-12b,Defence Suppliers: Innovation,Conservative,Mid Worcestershire,2016-12-12
